In [1]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true')
#                   .set('spark.driver.maxResultSize', '5G')\
#                   .set('spark.executor.memory', '1G')\
#                   .set('spark.driver.memory', '1G')\
                  

sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

Данные находятся в файле Train_Set_90621.csv
Amount Defaulted - эту переменную нужно удалить=)

    1) Начните с анализа баланса классов, пропусков, статистик при помощи DataFrame API
    2) Посомтрите статистики, заполните пропуски при помощи уже MLlib
    3) Соберите пайплайн, похожий на наш, где будет обработка данных, обучение моделей и все при помощи Spark
    4) Разбейте данные на train/test + реализуйте подбор параметров одним из способов спарка
    5) Cохраниет пайплайн на диск

In [2]:
from pyspark.sql.functions import col,isnan, when, count
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, IndexToString, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Imputer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import numpy as np
import matplotlib.pyplot as plt

### Read data

In [51]:
data = spark.read.csv('Train_Set_90621.csv', header=True, inferSchema=True)
data = data.drop('Amount Defaulted').withColumnRenamed('Default_Status', 'target')
data.printSchema()

root
 |-- Application ID: integer (nullable = true)
 |-- Bank Masked: string (nullable = true)
 |-- Bank Type: string (nullable = true)
 |-- Approved_Timestamp: string (nullable = true)
 |-- Name Masked: string (nullable = true)
 |-- Business Owner State: string (nullable = true)
 |-- Business_Industry_Type_Code: integer (nullable = true)
 |-- Approved_Year: integer (nullable = true)
 |-- New Business: integer (nullable = true)
 |-- Term: integer (nullable = true)
 |-- BankState: string (nullable = true)
 |-- Interest Rate: integer (nullable = true)
 |-- Employees: integer (nullable = true)
 |-- Gross Disbursed Amount: integer (nullable = true)
 |-- Term_years: integer (nullable = true)
 |-- Jobs Retained: integer (nullable = true)
 |-- Male to Female Employees Ratio: integer (nullable = true)
 |-- Expected Company Income: long (nullable = true)
 |-- Funds available with company: integer (nullable = true)
 |-- Gross_Apprv_Amount: integer (nullable = true)
 |-- Company Branch Code: inte

In [39]:
data.show()

+--------------+---------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+------+
|Application ID|    Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|target|
+--------------+---------------+----

In [4]:
# check Nan
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+--------------+-----------+---------+------------------+-----------+--------------------+---------------------------+-------------+------------+----+---------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+------+
|Application ID|Bank Masked|Bank Type|Approved_Timestamp|Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|target|
+--------------+-----------+---------+------------------+-----------+-

In [5]:
data.count()

195118

In [6]:
# check application_id duplicates
data.select(col('Application ID')).groupBy('Application ID').count().filter('count > 1').show()

+--------------+-----+
|Application ID|count|
+--------------+-----+
+--------------+-----+



#### Fill Nans

In [21]:
data.select(col('Jobs Generated')).groupBy('Jobs Generated').count().sort(col('count').desc()).show()

+--------------+------+
|Jobs Generated| count|
+--------------+------+
|             0|136854|
|             1| 13475|
|             2| 12221|
|             3|  6196|
|             4|  4427|
|             5|  4016|
|            10|  2512|
|             6|  2330|
|             8|  1671|
|             7|  1319|
|            15|  1081|
|            12|   954|
|            20|   933|
|             9|   749|
|            25|   490|
|            11|   465|
|            30|   451|
|            14|   421|
|            13|   376|
|            16|   362|
+--------------+------+
only showing top 20 rows



In [47]:
data = data.fillna({'Bank Masked': 'Other', 
                    'Bank Type': 'Private', 
                    'Business Owner State': 'Delhi', 
                    'New Business': 1,
                    'BankState': 'Delhi',
                    'Carry-forward Credit': 'N',
                    'Documents Provided': 'N'})\
            .withColumn("BankState", when(col('BankState') == 0, 'Delhi').otherwise(col('BankState')))\
            .withColumn("New Business", when(col('New Business') == 0, 2).otherwise(col('New Business')))

In [48]:
# drop na for some cols
drop_na_cols = ['Approved_Timestamp', 'Name Masked']

data = data.dropna(subset=drop_na_cols)
data.count()

194691

In [49]:
# fill na with mean for numeric cols
fill_mean_cols = ['Expected Company Income']

imputer = Imputer(inputCols=fill_mean_cols, outputCols=fill_mean_cols, strategy='mean')
imputer = imputer.fit(data)
data = imputer.transform(data)
data.count()

194691

In [11]:
# check Nan again
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+--------------+-----------+---------+------------------+-----------+--------------------+---------------------------+-------------+------------+----+---------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+------+
|Application ID|Bank Masked|Bank Type|Approved_Timestamp|Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|target|
+--------------+-----------+---------+------------------+-----------+-

### Create feature column as vector

In [50]:
index_cols = ['Bank Masked', 'Bank Type', 'Name Masked', 
              'Business Owner State', 'Business_Industry_Type_Code', 'New Business', 
              'BankState', 'Company Branch Code', 'City or Rural',
              'Carry-forward Credit', 'Documents Provided']

indexer_dict = {}

for column in index_cols:
    new_name = column.replace(" ", "_")+'_index'
#     print(f"{column}, {new_name}")
    indexer = StringIndexer(inputCol = column, outputCol = new_name)
    indexer = indexer.fit(data)
    data = indexer.transform(data)
    indexer_dict[new_name] = indexer

Py4JJavaError: An error occurred while calling o1793.fit.
: org.apache.spark.SparkException: Input column City or RuralCarry-forward Credit does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringInde

In [35]:
# create sin/cos features from datetime col
data = data.withColumn('Approved_Timestamp', F.to_date(col("Approved_Timestamp"),"dd-MM-yyyy"))\
    .withColumn('Approved_year', F.year('Approved_Timestamp'))\
    .withColumn('Approved_month', F.month('Approved_Timestamp'))\
    .withColumn('Approved_dayofmonth', F.dayofmonth('Approved_Timestamp'))\
    .withColumn('Approved_dayofweek', F.dayofweek('Approved_Timestamp'))\
    .withColumn('cos_approved_month', F.cos(F.col('Approved_month') * 2 * np.pi / 12.))\
    .withColumn('sin_approved_month', F.sin(F.col('Approved_month') * 2 * np.pi / 12.))\
    .withColumn('cos_approved_dayofmonth', F.cos(F.col('Approved_dayofmonth') * 2 * np.pi / 31.))\
    .withColumn('sin_approved_dayofmonth', F.sin(F.col('Approved_dayofmonth') * 2 * np.pi / 31.))\
    .withColumn('cos_approved_dayofweek', F.cos(F.col('Approved_dayofweek') * 2 * np.pi / 7.))\
    .withColumn('sin_approved_dayofweek', F.sin(F.col('Approved_dayofweek') * 2 * np.pi / 7.))

+------------------+-------------+--------------+-------------------+------------------+--------------------+--------------------+-----------------------+-----------------------+----------------------+----------------------+
|Approved_Timestamp|Approved_year|Approved_month|Approved_dayofmonth|Approved_dayofweek|  cos_approved_month|  sin_approved_month|cos_approved_dayofmonth|sin_approved_dayofmonth|cos_approved_dayofweek|sin_approved_dayofweek|
+------------------+-------------+--------------+-------------------+------------------+--------------------+--------------------+-----------------------+-----------------------+----------------------+----------------------+
|        2005-10-12|         2005|            10|                 12|                 4|  0.5000000000000001| -0.8660254037844386|    -0.7587581226927909|     0.6513724827222223|   -0.9009688679024191|    -0.433883739117558|
|        2006-11-21|         2006|            11|                 21|                 3|  0.86602540

In [ ]:
ohe_encoder = OneHotEncoder(inputCols = list(indexer_dict.keys()),
                        outputCols = [col[:-6]+'_vec' for col in list(indexer_dict.keys())])
ohe_encoder = ohe_encoder.fit(data)
data = ohe_encoder.transform(data)

In [61]:
data.select('Bank Masked', 'Bank_Masked_index', 'Bank_Masked_vec').show()

+---------------+-----------------+--------------------+
|    Bank Masked|Bank_Masked_index|     Bank_Masked_vec|
+---------------+-----------------+--------------------+
|  Bank_Name_316|           1260.0|(42528,[1260],[1.0])|
| Bank_Name_1062|            563.0| (42528,[563],[1.0])|
|   Bank_Name_39|             21.0|  (42528,[21],[1.0])|
|    Bank_Name_3|              1.0|   (42528,[1],[1.0])|
|  Bank_Name_126|             33.0|  (42528,[33],[1.0])|
| Bank_Name_1281|           1419.0|(42528,[1419],[1.0])|
|   Bank_Name_33|             10.0|  (42528,[10],[1.0])|
|   Bank_Name_33|             10.0|  (42528,[10],[1.0])|
|Bank_Name_12778|           7154.0|(42528,[7154],[1.0])|
|   Bank_Name_17|              5.0|   (42528,[5],[1.0])|
|Bank_Number_346|            186.0| (42528,[186],[1.0])|
|   Bank_Name_47|             24.0|  (42528,[24],[1.0])|
| Bank_Number_93|             50.0|  (42528,[50],[1.0])|
|  Bank_Name_666|            248.0| (42528,[248],[1.0])|
|    Bank_Name_3|              

In [86]:
# assemble features in vector format
feature_columns = ['Approved_year', 'Term', 'Interest Rate', 'Employees', 'Gross Disbursed Amount', 
                   'Term_years', 'Jobs Retained', 'Male to Female Employees Ratio', 'Expected Company Income', 
                   'Funds available with company', 'Gross_Apprv_Amount', 'Jobs Generated', 'Balance Left', 'Final_Appved_Amount']\
                    + ['cos_approved_month', 'sin_approved_month', 
                       'cos_approved_dayofmonth', 'sin_approved_dayofmonth', 
                       'cos_approved_dayofweek', 'sin_approved_dayofweek']\
                    + vect_columns

df_va = VectorAssembler(inputCols = feature_columns, outputCol = 'features')
data = df_va.transform(data)

In [83]:
# total features
len(feature_columns)

31

In [87]:
data.show()

+--------------+---------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+------+--------------+-------------------+------------------+--------------------+--------------------+-----------------------+-----------------------+----------------------+----------------------+-----------------+---------------+-----------------+--------------------------+---------------------------------+------------------+---------------+-------------------------+-------------------+--------------------------+------------------------+--------------------+-------------+--------------------+------------------------+--

In [91]:
# train/test split
train, test = data.randomSplit([0.7, 0.3], seed=135468)

## Some models

In [5]:
lrModel = LogisticRegression(featuresCol='features', labelCol='target', predictionCol='prediction',
                        maxIter=10, probabilityCol='proba')

lrModel = lrModel.fit(test)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\Mr Rozum\miniconda3\envs\gru_torch\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Mr Rozum\AppData\Local\Temp\ipykernel_26540\3793522769.py", line 4, in <module>
    lrModel = lrModel.fit(test)
  File "C:\Users\Mr Rozum\miniconda3\envs\gru_torch\lib\site-packages\pyspark\ml\base.py", line 161, in fit
    return self._fit(dataset)
  File "C:\Users\Mr Rozum\miniconda3\envs\gru_torch\lib\site-packages\pyspark\ml\wrapper.py", line 335, in _fit
    java_model = self._fit_java(dataset)
  File "C:\Users\Mr Rozum\miniconda3\envs\gru_torch\lib\site-packages\pyspark\ml\wrapper.py", line 332, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "C:\Users\Mr Rozum\miniconda3\envs\gru_torch\lib\site-packages\py4j\java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  Fil

ConnectionRefusedError: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение

In [4]:
print(">>> Model parameters:")
print(">>> Coefficients: " + str(lrModel.coefficients))
print(">>> Intercept: " + str(lrModel.intercept))
print(f'>>> ROC_AUC = {lrModel.summary.areaUnderROC}')

>>> Model parameters:


NameError: name 'lrModel' is not defined

In [ ]:
# ROC curve
roc = trainingSummary.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

In [ ]:
# precision-recall curve
pr = trainingSummary.pr.toPandas()
plt.plot(pr['recall'],pr['precision'])
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.show()

## Проверять отсюда

In [3]:
# preprocess pipeline

# ---------
# read data
data = spark.read.csv('Train_Set_90621.csv', header=True, inferSchema=True)
data = data.drop('Amount Defaulted').withColumnRenamed('Default_Status', 'target')

# --------------------------------
# process Nan and small categories
data = data.fillna({'Bank Masked': 'Other', 
                    'Bank Type': 'Private', 
                    'Business Owner State': 'Delhi', 
                    'New Business': 1,
                    'BankState': 'Delhi',
                    'Carry-forward Credit': 'N',
                    'Documents Provided': 'N'})\
            .withColumn("BankState", when(col('BankState') == 0, 'Delhi').otherwise(col('BankState')))\
            .withColumn("New Business", when(col('New Business') == 0, 2).otherwise(col('New Business')))

# ---------------------
# drop na for some cols
drop_na_cols = ['Approved_Timestamp', 'Name Masked']
data = data.dropna(subset=drop_na_cols)

# ----------------------------------
# fill na with mean for numeric cols
fill_mean_cols = ['Expected Company Income']
imputer = Imputer(inputCols=fill_mean_cols, outputCols=fill_mean_cols, strategy='mean')
imputer = imputer.fit(data)
data = imputer.transform(data)

# -----------------------------
# create sin/col cols from date
data = data.withColumn('Approved_Timestamp', F.to_date(col("Approved_Timestamp"),"dd-MM-yyyy"))\
    .withColumn('Approved_year', F.year('Approved_Timestamp'))\
    .withColumn('Approved_month', F.month('Approved_Timestamp'))\
    .withColumn('Approved_dayofmonth', F.dayofmonth('Approved_Timestamp'))\
    .withColumn('Approved_dayofweek', F.dayofweek('Approved_Timestamp'))\
    .withColumn('cos_approved_month', F.cos(F.col('Approved_month') * 2 * np.pi / 12.))\
    .withColumn('sin_approved_month', F.sin(F.col('Approved_month') * 2 * np.pi / 12.))\
    .withColumn('cos_approved_dayofmonth', F.cos(F.col('Approved_dayofmonth') * 2 * np.pi / 31.))\
    .withColumn('sin_approved_dayofmonth', F.sin(F.col('Approved_dayofmonth') * 2 * np.pi / 31.))\
    .withColumn('cos_approved_dayofweek', F.cos(F.col('Approved_dayofweek') * 2 * np.pi / 7.))\
    .withColumn('sin_approved_dayofweek', F.sin(F.col('Approved_dayofweek') * 2 * np.pi / 7.))

# ----------------------
# category cols to index
index_cols = ['Bank Masked', 'Bank Type', 'Name Masked', 
              'Business Owner State', 'Business_Industry_Type_Code', 'New Business', 
              'BankState', 'Company Branch Code', 'City or Rural',
              'Carry-forward Credit', 'Documents Provided']

res_index_cols = [col.replace(" ", "_")+'_index' for col in index_cols]
vect_columns = [col[:-6]+'_vec' for col in res_index_cols]

Bank_Masked_index = StringIndexer(inputCol = 'Bank Masked', outputCol = 'Bank_Masked_index', handleInvalid='keep')
Bank_Type_index = StringIndexer(inputCol = 'Bank Type', outputCol = 'Bank_Type_index', handleInvalid='keep')
Name_Masked_index = StringIndexer(inputCol = 'Name Masked', outputCol = 'Name_Masked_index', handleInvalid='keep')
Business_Owner_State_index = StringIndexer(inputCol = 'Business Owner State', outputCol = 'Business_Owner_State_index', handleInvalid='keep')
Business_Industry_Type_Code_index = StringIndexer(inputCol = 'Business_Industry_Type_Code', outputCol = 'Business_Industry_Type_Code_index', handleInvalid='keep')
New_Business_index = StringIndexer(inputCol = 'New Business', outputCol = 'New_Business_index', handleInvalid='keep')
BankState_index = StringIndexer(inputCol = 'BankState', outputCol = 'BankState_index', handleInvalid='keep')
Company_Branch_Code_index = StringIndexer(inputCol = 'Company Branch Code', outputCol = 'Company_Branch_Code_index', handleInvalid='keep')
City_or_Rural_index = StringIndexer(inputCol = 'City or Rural', outputCol = 'City_or_Rural_index', handleInvalid='keep')
Carry_forward_Credit_index = StringIndexer(inputCol = 'Carry-forward Credit', outputCol = 'Carry-forward_Credit_index', handleInvalid='keep')
Documents_Provided_index = StringIndexer(inputCol = 'Documents Provided', outputCol = 'Documents_Provided_index', handleInvalid='keep')

# indexer_dict = {}
# for column in index_cols:
#     new_name = column.replace(" ", "_")+'_index'
# #     print(f"{column}, {new_name}")
#     indexer = StringIndexer(inputCol = column, outputCol = new_name)
#     indexer = indexer.fit(data)
#     data = indexer.transform(data)
#     indexer_dict[new_name] = indexer

# ------------
# ohe encoding
# vect_columns = [col[:-6]+'_vec' for col in list(indexer_dict.keys())]
ohe_encoder = OneHotEncoder(inputCols = res_index_cols,
                        outputCols = vect_columns)
# ohe_encoder = ohe_encoder.fit(data)
# data = ohe_encoder.transform(data)

# ----------------------------------
# assemble features in vector format
feature_columns = ['Approved_year', 'Term', 'Interest Rate', 'Employees', 'Gross Disbursed Amount', 
                   'Term_years', 'Jobs Retained', 'Male to Female Employees Ratio', 'Expected Company Income', 
                   'Funds available with company', 'Gross_Apprv_Amount', 'Jobs Generated', 'Balance Left', 'Final_Appved_Amount']\
                    + ['cos_approved_month', 'sin_approved_month', 
                       'cos_approved_dayofmonth', 'sin_approved_dayofmonth', 
                       'cos_approved_dayofweek', 'sin_approved_dayofweek']\
                    + vect_columns

vector_assembler = VectorAssembler(inputCols = feature_columns, outputCol = 'features')

rfModel = RandomForestClassifier(featuresCol = 'features', labelCol = 'target', predictionCol='prediction',probabilityCol='proba', 
                                  numTrees=10,  maxDepth=7, seed=145795, subsamplingRate=0.5)

# data = df_va.transform(data)



In [ ]:
pipeline = Pipeline(stages=[
                            Bank_Masked_index,
                            Bank_Type_index,
                            Name_Masked_index,
                            Business_Owner_State_index,
                            Business_Industry_Type_Code_index,
                            New_Business_index,
                            BankState_index,
                            Company_Branch_Code_index,
                            City_or_Rural_index,
                            Carry_forward_Credit_index,
                            Documents_Provided_index,
                            ohe_encoder,
                            vector_assembler,
                            rfModel
])

paramGrid = ParamGridBuilder()\
    .addGrid(rfModel.maxDepth, [5, 3]) \
    .build()

tvs = TrainValidationSplit(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                                  labelCol='target', metricName='areaUnderROC'),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.7)

train, test = data.randomSplit([0.7, 0.3], seed=135468)

model = tvs.fit(train)

rf_test_pred = model.transform(test)

rf_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol = 'target')
print("Test Area Under ROC: " + str(rf_evaluator.evaluate(rf_test_pred, {rf_evaluator.metricName: "areaUnderROC"})))

In [ ]:
tvs.write().save('model')